# qecsim demos

## Simulating error correction with a toric stabilizer code
This demo shows verbosely how to simulate one error correction run. 

| For normal use, the code in this demo is encapsulated in the function:
| `qecsim.app.run_once(code, error_model, decoder, error_probability)`,
| and the simulation of many error correction runs is encapsulated in the function:
| `qecsim.app.run(code, error_model, decoder, error_probability, max_runs, max_failures)`.

Notes:

* Operators can be visualised in binary symplectic form (bsf) or Pauli form, e.g. `[1 1 0|0 1 0] = XYI`.
* The binary symplectic product is denoted by $\odot$ and defined as $A \odot B \equiv A \Lambda B \bmod 2$ where $\Lambda = \left[\begin{matrix} 0 & I \\ I & 0 \end{matrix}\right]$.
* Binary addition is denoted by $\oplus$ and defined as addition modulo 2, or equivalently exclusive-or.

### Initialise the models

In [18]:
%run qsu.ipynb  # color-printing functions
import numpy as np
from qecsim import paulitools as pt
from qecsim.models.generic import DepolarizingErrorModel
from qecsim.models.toric import ToricCode, ToricMWPMDecoder

# initialise models
my_code = ToricCode(3, 4)
my_error_model = DepolarizingErrorModel()
my_decoder = ToricMWPMDecoder()
# print models
print(my_code)
print(my_error_model)
print(my_decoder)

ToricCode(3, 4)
DepolarizingErrorModel()
ToricMWPMDecoder()


### Generate a random error

In [19]:
# set physical error probability to 10%
error_probability = 0.05
# seed random number generator for repeatability
rng = np.random.default_rng(59)

# error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
qsu.print_pauli('error:\n{}'.format(my_code.new_pauli(error)))

In [16]:
hadamard_mat=np.zeros((2,3,4))
hadamard_mat.shape

(2, 3, 4)

In [20]:
for i,j,k in np.ndindex(hadamard_mat.shape):
    print(i,j,k,i*12+j*4+k)

0 0 0 0
0 0 1 1
0 0 2 2
0 0 3 3
0 1 0 4
0 1 1 5
0 1 2 6
0 1 3 7
0 2 0 8
0 2 1 9
0 2 2 10
0 2 3 11
1 0 0 12
1 0 1 13
1 0 2 14
1 0 3 15
1 1 0 16
1 1 1 17
1 1 2 18
1 1 3 19
1 2 0 20
1 2 1 21
1 2 2 22
1 2 3 23


In [24]:
_,nrows,ncols=hadamard_mat.shape
hadamard_vec=np.zeros(2*np.prod((nrows,ncols)))
for i,j,k in np.ndindex(hadamard_mat.shape):
    hadamard_mat[0,j,k]=1

for i,j,k in np.ndindex(hadamard_mat.shape):
    hadamard_vec[i*nrows*ncols+j*ncols+k]=hadamard_mat[i,j,k]
    print(i,j,k,hadamard_mat[i,j,k])
    


0 0 0 1.0
0 0 1 1.0
0 0 2 1.0
0 0 3 1.0
0 1 0 1.0
0 1 1 1.0
0 1 2 1.0
0 1 3 1.0
0 2 0 1.0
0 2 1 1.0
0 2 2 1.0
0 2 3 1.0
1 0 0 0.0
1 0 1 0.0
1 0 2 0.0
1 0 3 0.0
1 1 0 0.0
1 1 1 0.0
1 1 2 0.0
1 1 3 0.0
1 2 0 0.0
1 2 1 0.0
1 2 2 0.0
1 2 3 0.0


### Evaluate the syndrome
The syndrome is a binary array indicating the stabilizers with which the error does not commute. It is calculated as $syndrome = error \odot stabilisers^T$.

In [3]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

### Find a recovery operation
In this case, the recovery operation is found by a minimum weight perfect matching decoder that finds the recovery operation as follows:

* The syndrome is resolved to plaquettes using: `ToricCode.syndrome_to_plaquette_indices`.
* A graph between plaquettes is built with weights given by: `ToricMWPMDecoder.distance`.
* A MWPM algorithm is used to match plaquettes into pairs.
* A recovery operator is constructed by applying the shortest path between matching plaquette pairs using:
  `ToricPauli.path`.

In [4]:
# recovery: best match recovery operation based on decoder
recovery = my_decoder.decode(my_code, syndrome)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

As a sanity check, we expect $recovery \oplus error$ to commute with all stabilizers, i.e. $(recovery \oplus error) \odot stabilisers^T = 0$.

In [5]:
# check recovery ^ error commutes with stabilizers (by construction)
print(pt.bsp(recovery ^ error, my_code.stabilizers.T))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Visualise $recovery \oplus error$
Just out of curiosity, we can see what $recovery \oplus error$ looks like. If successful, it should be a product of stabilizer plaquette / vertex operators.

In [6]:
# print recovery ^ error (out of curiosity)
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

### Test if the recovery operation is successful
The recovery operation is successful iff $recovery \oplus error$ commutes with all logical operators, i.e. $(recovery \oplus error) \odot logicals^T = 0.$

In [7]:
# success iff recovery ^ error commutes with logicals
print(pt.bsp(recovery ^ error, my_code.logicals.T))

[1 0 0 0]


Note: The decoder is not guaranteed to find a successful recovery operation. The toric 5 x 5 code has distance $d = 5$ so we can only guarantee to correct errors up to weight $(d - 1)/2=2$.

### Equivalent code in single call
The above demo is equivalent to the following code.

In [8]:
# repeat demo in single call
from qecsim import app
print(app.run_once(my_code, my_error_model, my_decoder, error_probability))

{'error_weight': 4, 'success': True}


In [171]:
def dic(Nx, Ny):
    v = np.zeros((Nx, Ny),dtype=int)
    for i in range(0,Nx):
        for j in range(0,Ny):
            v[i,j] = int(i+j*Nx)
    return v

dic(4,4)

array([[ 0,  4,  8, 12],
       [ 1,  5,  9, 13],
       [ 2,  6, 10, 14],
       [ 3,  7, 11, 15]])

In [285]:
def dicX(Nx, Ny,virX1,virX2):
    v = np.zeros((Nx+2, Ny+2),dtype=int)
    for i in range(0,Nx+1):
        for j in range(0,Ny+2):
            if j==1:
                v[i,j]= virX1
            elif j==Ny+1:
                v[i,j]= virX2
            else:
                v[i,j] = round(i+(j-2)*(Nx+1))
    return v

def dicZ(Nx, Ny,virZ1,virZ2):
    v = np.zeros((Nx+2, Ny+2),dtype=int)
    for i in range(0,Nx+2):
        for j in range(1,Ny+1):
            if i==0:
                v[i,j] = virZ1
            elif i==Nx+1:
                v[i,j] = virZ2
            else:
                v[i,j] = round(i-1+(j-1)*Nx)
    return v


Nx=3
Ny=3
virZ1= Nx*Ny+(Nx+1)*(Ny-1)
virZ2= Nx*Ny+(Nx+1)*(Ny-1)+1
virX1= Nx*Ny+(Nx+1)*(Ny-1)+2
virX2= Nx*Ny+(Nx+1)*(Ny-1)+3

print(dicZ(Nx, Ny,virZ1,virZ2).T)
dicX(Nx, Ny,virX1,virX2).T

[[ 0  0  0  0  0]
 [17  0  1  2 18]
 [17  3  4  5 18]
 [17  6  7  8 18]
 [ 0  0  0  0  0]]


array([[-8, -7, -6, -5,  0],
       [19, 19, 19, 19,  0],
       [ 0,  1,  2,  3,  0],
       [ 4,  5,  6,  7,  0],
       [20, 20, 20, 20,  0]])

In [311]:
vX

array([[19, -1,  3, 20, 11],
       [19,  0,  4, 20, 12],
       [19,  1,  5, 20, 13],
       [19,  2,  6, 20, 14],
       [ 0,  0,  0,  0,  0]])

In [334]:
def dicX(Nx, Ny,virX1,virX2):
    v = np.zeros((Nx+2, Ny+2),dtype=int)
    for i in range(0,Nx+2):
        for j in range(0,Ny+2):
            if j==0:
                v[i,j]= virX1
            elif j==Ny:
                v[i,j]= virX2
            else:
                v[i,j] = round(i-1+(j-1)*(Nx+1))
    return v

def dicZ(Nx, Ny,virZ1,virZ2):
    v = np.zeros((Nx+2, Ny+2),dtype=int)
    for i in range(0,Nx+2):
        for j in range(1,Ny+1):
            if i==0:
                v[i,j] = virZ1
            elif i==Nx+1:
                v[i,j] = virZ2
            else:
                v[i,j] = round(i-1+(j-1)*Nx)
    return v


import networkx as nx


pX=0.2
pZ=0.2
pY=0.1

Nx=3
Ny=3
virZ1= Nx*Ny+(Nx+1)*(Ny-1)
virZ2= Nx*Ny+(Nx+1)*(Ny-1)+1
virX1= Nx*Ny+(Nx+1)*(Ny-1)+2
virX2= Nx*Ny+(Nx+1)*(Ny-1)+3

vZ=dicZ(Nx, Ny,virZ1,virZ2)
vX=dicX(Nx, Ny,virX1,virX2)
Nx_X=Nx+1
Ny_X=Ny-1

#error probabilities
pI = 1-pX-pY-pZ

#weights
wZ = abs(np.log(pZ / pI))
wX = abs(np.log(pX / pI))

Graph_Zerr= nx.Graph()
Graph_Xerr= nx.Graph()

X_logical1_Graph_Zerr=np.array([], dtype=int)
Z_logical1_Graph_Xerr=np.array([], dtype=int)


X_syndrome_num=Nx*Ny+2  #total number of X syndromes including the virtual ones 
Z_syndrome_num=(Nx+1)*(Ny-1)+2  #total number of Z syndromes including the virtual ones 
total_syndrome_num=X_syndrome_num+Z_syndrome_num

#construct the Z lattice
id=0
for i in range(1,Nx+1):
    for j in range(1,Ny+1):
        if j==Ny:
            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id, weight=wZ, error_probability=pZ)  
            Graph_Xerr.add_edge(vX[i+1,j-1], vX[i+1,j], qubit_id=id, weight=wX, error_probability=pX) 
            id+=1
        else:
            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id, weight=wZ, error_probability=pZ)  
            Graph_Xerr.add_edge(vX[i+1,j-1], vX[i+1,j], qubit_id=id, weight=wX, error_probability=pX) 
            id+=1
            Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id, weight=wZ, error_probability=pZ) 
            Graph_Xerr.add_edge(vX[i,j],vX[i+1,j],qubit_id=id, weight=wX, error_probability=pX)
            id+=1
        if i==1:
            Graph_Zerr.add_edge(vZ[i-1,j],vZ[i,j],qubit_id=id, weight=wZ, error_probability=pZ)  
            Graph_Xerr.add_edge(vX[i,j-1], vX[i,j], qubit_id=id, weight=wX, error_probability=pX) 
        print(i,j,vZ[i,j],vZ[i+1,j],vX[i+1,j-1], vX[i+1,j])
        print(i,j,vZ[i,j],vZ[i,j+1],vX[i,j-1], vX[i,j])


        
Z_logical_qubit_num = Nx+1     #number of qubits of the lowest y-line for Z logical
X_logical_qubit_num = Ny       #number of qubits of the leftmost x-line for X logical



1 1 0 1 19 1
1 1 0 3 19 0
1 2 3 4 1 5
1 2 3 6 0 4
1 3 6 7 5 20
1 3 6 0 4 20
2 1 1 2 19 2
2 1 1 4 19 1
2 2 4 5 2 6
2 2 4 7 1 5
2 3 7 8 6 20
2 3 7 0 5 20
3 1 2 18 19 3
3 1 2 5 19 2
3 2 5 18 3 7
3 2 5 8 2 6
3 3 8 18 7 20
3 3 8 0 6 20


In [313]:
Graph_Zerr.edges

EdgeView([(0, 1), (0, 3), (0, 17), (1, 2), (1, 4), (3, 4), (3, 6), (3, 17), (17, 6), (17, 18), (4, 5), (4, 7), (6, 7), (7, 8), (2, 18), (2, 5), (5, 18), (5, 8), (8, 18)])

In [316]:
Graph_Xerr.edges

EdgeView([(19, 1), (19, 0), (19, 2), (19, 3), (19, 20), (1, 0), (1, 5), (1, 2), (0, 4), (5, 4), (5, 20), (5, 6), (4, 20), (20, 6), (20, 7), (2, 6), (2, 3), (6, 7), (3, 7)])

In [240]:
def dicX(Nx, Ny):
    v = np.zeros((Nx, Ny),dtype=int)
    for i in range(0,Nx):
        for j in range(0,Ny+2):
            if j==0:
                v[i,j]= virX1
            elif j==Ny+1
                v[i,j]= virX2
            else:
                v[i,j] = round(i+j*Nx)
    return v

def dicZ(Nx, Ny,virZ1,virZ2):
    v = np.zeros((Nx, Ny),dtype=int)
    for i in range(0,Nx+2):
        for j in range(0,Ny):
            if i==0:
                v[i,j] = virZ1
            elif i==Nx+2:
                v[i,j] = virZ2
            else:
                v[i,j] = round(i+j*Nx)
    return v



import networkx as nx


pX=0.2
pZ=0.2
pY=0.1

Nx=3
Ny=3

vZ=dic(Nx, Ny)
Nx_X=Nx+1
Ny_X=Ny-1
vX=dic(Nx_X, Ny_X)

#error probabilities
pI = 1-pX-pY-pZ

#weights
wZ = abs(np.log(pZ / pI))
wX = abs(np.log(pX / pI))

Graph_Zerr= nx.Graph()
Graph_Xerr= nx.Graph()

X_logical1_Graph_Zerr=np.array([], dtype=int)
Z_logical1_Graph_Xerr=np.array([], dtype=int)

virZ1= Nx*Ny+(Nx+1)*(Ny-1)
virZ2= Nx*Ny+(Nx+1)*(Ny-1)+1
virX1= Nx*Ny+(Nx+1)*(Ny-1)+2
virX2= Nx*Ny+(Nx+1)*(Ny-1)+3

X_syndrome_num=Nx*Ny+2  #total number of X syndromes including the virtual ones 
Z_syndrome_num=(Nx+1)*(Ny-1)+2  #total number of Z syndromes including the virtual ones 
total_syndrome_num=X_syndrome_num+Z_syndrome_num

#construct the Z lattice
id=0
for i in range(Nx):
    for j in range(Ny):
        #bulk
        if (i!=0 and i!=Nx-1 and j!=0 and j!=Ny-1):
            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id, weight=wZ, error_probability=pZ)  
            Graph_Xerr.add_edge(vX[i+1,j-1], vX[i+1,j], qubit_id=id, weight=wX, error_probability=pX) 
            id+=1
            Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id, weight=wZ, error_probability=pZ) 
            Graph_Xerr.add_edge(vX[i,j],vX[i+1,j],qubit_id=id, weight=wX, error_probability=pX)
            id+=1
        
        #x=0
        if i==0:
            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id, weight=wZ, error_probability=pZ)
            if j==0:
                Graph_Xerr.add_edge(virX1,vX[i+1,j],qubit_id=id, weight=wX, error_probability=pX) 
            elif j==Ny-1:
                Graph_Xerr.add_edge(vX[i+1,j-1],virX2,qubit_id=id, weight=wX, error_probability=pX) 
            else:
                Graph_Xerr.add_edge(vX[i+1,j-1],vX[i+1,j], qubit_id=id, weight=wX, error_probability=pX) 
            id+=1
            
            if j!=Ny-1:
                Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id, weight=wZ, error_probability=pZ) 
                Graph_Xerr.add_edge(vX[i,j],vX[i+1,j],qubit_id=id, weight=wX, error_probability=pX)
            id+=1
            
            Graph_Zerr.add_edge(virZ1,vZ[i,j],qubit_id=id, weight=wZ, error_probability=pZ) 
            if j==0:
                Graph_Xerr.add_edge(virX1,vX[i,j],qubit_id=id, weight=wX, error_probability=pX) 
            elif j==Ny-1:
                Graph_Xerr.add_edge(vX[i,j-1],virX2,qubit_id=id, weight=wX, error_probability=pX) 
            else:
                Graph_Xerr.add_edge(vX[i,j-1],vX[i,j],qubit_id=id, weight=wX, error_probability=pX) 
            id+=1


        #x=Nx-1
        if i==Nx-1:
            if j!=Ny-1:
                Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id, weight=wZ, error_probability=pZ) 
                Graph_Xerr.add_edge(vX[i,j],vX[i+1,j],qubit_id=id, weight=wX, error_probability=pX)
                id+=1
            Graph_Zerr.add_edge(vZ[i,j],virZ2,qubit_id=id, weight=wZ, error_probability=pZ) 
            if j==0:
                Graph_Xerr.add_edge(virX1,vX[i+1,j], qubit_id=id, weight=wX, error_probability=pX) 
            elif j==Ny-1:
                Graph_Xerr.add_edge(vX[i+1,j-1],virX2, qubit_id=id, weight=wX, error_probability=pX)             
            else:
                Graph_Xerr.add_edge(vX[i+1,j-1],vX[i+1,j], qubit_id=id, weight=wX, error_probability=pX)                 
            id+=1
            
        #y=0
        if (j==0 and i!=0 and i!=Nx-1):
            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id, weight=wZ, error_probability=pZ)  
            Graph_Xerr.add_edge(virX1, vX[i+1,j], qubit_id=id, weight=wX, error_probability=pX) 
            id+=1
            Graph_Zerr.add_edge(vZ[i,j],vZ[i,j+1],qubit_id=id, weight=wZ, error_probability=pZ) 
            Graph_Xerr.add_edge(vX[i,j],vX[i+1,j],qubit_id=id, weight=wX, error_probability=pX)
            id+=1
        
        #y=Ny-1
        if (j==Ny-1 and i!=0 and i!=Nx-1):
            Graph_Zerr.add_edge(vZ[i,j],vZ[i+1,j],qubit_id=id, weight=wZ, error_probability=pZ)  
            Graph_Xerr.add_edge(vX[i+1,j-1], virX2, qubit_id=id, weight=wX, error_probability=pX) 
            id+=1


Graph_Zerr.add_edge(virZ1,virZ2,qubit_id=set(), weight=0, error_probability=0)  
Graph_Xerr.add_edge(virX1,virX2,qubit_id=set(), weight=0, error_probability=0) 

        
Z_logical_qubit_num = Nx+1     #number of qubits of the lowest y-line for Z logical
X_logical_qubit_num = Ny       #number of qubits of the leftmost x-line for X logical



In [241]:
Graph_Zerr.edges

EdgeView([(0, 1), (0, 3), (0, 17), (1, 2), (1, 4), (3, 4), (3, 6), (3, 17), (17, 6), (17, 18), (4, 5), (4, 7), (6, 7), (7, 8), (2, 5), (2, 18), (5, 8), (5, 18), (8, 18)])

In [281]:
Graph_Xerr.edges

EdgeView([(0, 0), (0, 19), (0, 3), (0, 2), (0, 20), (19, 3), (19, 20), (3, 6), (2, 1), (2, 5), (5, 4), (5, 8), (20, 20)])

In [242]:
 for i in range(0,Nx):
    for j in range(0,Ny):
        if vZ[i,j] in range(0,Nx*Ny,Nx):
            X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[i,j]][vZ[(i + 1)% Nx,j]]['qubit_id'])
        if vX[i,j] in range(0,Nx):
            Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,j]][vX[i,(j+1)%Ny]]['qubit_id'])
        if vZ[i,j] in range(0,Nx):
            X_logical2_Graph_Zerr=np.append(X_logical2_Graph_Zerr,Graph_Zerr[vZ[i,j]][vZ[i,(j+1)%Ny]]['qubit_id'])
        if vX[i,j] in range(0,Nx*Ny,Nx):
            Z_logical2_Graph_Xerr=np.append(Z_logical2_Graph_Xerr,Graph_Xerr[vX[i,j]][vX[(i + 1)% Nx,j]]['qubit_id'])

IndexError: index 2 is out of bounds for axis 1 with size 2

In [136]:
G = nx.Graph([(1, 2, {"color": "yellow"})])
G[1][2]["color"]

'yellow'

In [341]:
for i in range(1,Nx+1):
    for j in range(1,Ny+1):
        if vZ[i,j]in range(0,Nx*Ny,Nx):
            X_logical1_Graph_Zerr=np.append(X_logical1_Graph_Zerr,Graph_Zerr[vZ[i,j]][virZ1]['qubit_id'])

for i in range(1,Nx+2):
    for j in range(1,Ny):
        if vX[i,j]in range(0,Nx+1):
             Z_logical1_Graph_Xerr=np.append(Z_logical1_Graph_Xerr,Graph_Xerr[vX[i,j]][virX1]['qubit_id'])

In [342]:
Z_logical1_Graph_Xerr

array([ 2,  0,  5, 10])

In [344]:
X_logical1_Graph_Zerr

array([2, 4, 5])

In [345]:
for i,j in Graph_Xerr.edges:
    print(i,j,Graph_Xerr[i][j]['qubit_id'])

19 1 0
19 0 2
19 2 5
19 3 10
1 0 1
1 5 2
1 2 6
0 4 4
5 4 3
5 20 4
5 6 8
4 20 5
20 6 9
20 7 14
2 6 7
2 3 11
6 7 13
3 7 12


In [346]:
for i,j in Graph_Zerr.edges:
    print(i,j,Graph_Zerr[i][j]['qubit_id'])

0 1 0
0 3 1
0 17 2
1 2 5
1 4 6
3 4 2
3 6 3
3 17 4
17 6 5
4 5 7
4 7 8
6 7 4
7 8 9
2 18 10
2 5 11
5 18 12
5 8 13
8 18 14


In [ ]:
Graph_Zerr.nodes[len(Graph_Zerr.nodes)-1]['is_boundary'] = True
Graph_Zerr.nodes[len(Graph_Zerr.nodes)-2]['is_boundary'] = True
Graph_Xerr.nodes[len(Graph_Xerr.nodes)-1]['is_boundary'] = True   
Graph_Xerr.nodes[len(Graph_Xerr.nodes)-2]['is_boundary'] = True   

In [364]:
Graph_Zerr.nodes[virZ1]['is_boundary'] = True
Graph_Zerr.nodes[virZ2]['is_boundary'] = True
Graph_Xerr.nodes[virX1]['is_boundary'] = True   
Graph_Xerr.nodes[virX2]['is_boundary'] = True   

In [221]:
Z_logical1_Graph_Xerr

array([ 3,  1,  0,  6, 12])

In [368]:
Graph_Zerr[0][1]['qubit_id']>-1

True

In [223]:
x=Graph_Zerr.nodes;x[10]

KeyError: 10

KeyError: 10

In [93]:
len(Graph_Zerr.nodes)-2

25

In [224]:
from pymatching import Matching
import networkx as nx
import numpy as np

p = 0.2
g = nx.Graph()
g.add_edge(0, 1, qubit_id=0, weight=np.log((1-p)/p), error_probability=p)
g.add_edge(1, 2, qubit_id=1, weight=np.log((1-p)/p), error_probability=p)
g.add_edge(2, 3, qubit_id=2, weight=np.log((1-p)/p), error_probability=p)
g.add_edge(3, 4, qubit_id=3, weight=np.log((1-p)/p), error_probability=p)
g.add_edge(4, 5, qubit_id=4, weight=np.log((1-p)/p), error_probability=p)

In [225]:
p2 = 0.12
g.add_edge(2, 4, qubit_id={2, 3}, weight=np.log((1-p2)/p2), error_probability=p2)
g.nodes[0]['is_boundary'] = True
g.nodes[5]['is_boundary'] = True

In [226]:
g[0][1]

{'qubit_id': 0, 'weight': 1.3862943611198906, 'error_probability': 0.2}

In [232]:
x=g.nodes;x[3]

{}

In [36]:
[vX,vZ]=dic_pbc(Nx, Ny)

#error probabilities
pI = 1-pX-pY-pZ

#weights
wZ = abs(np.log(pZ / pI))
wX = abs(np.log(pX / pI))

Graph_Zerr= nx.Graph()
Graph_Xerr= nx.Graph()

id=0
twist=0
if(twist==0):
    for i in range(0,Nx):
        for j in range(0,Ny):
            Graph_Zerr.add_edge(vZ[i][j], vZ[(i + 1)% Nx][j], qubit_id=id, weight=wZ, error_probability=pZ)
            Graph_Xerr.add_edge(vX[(i+1)%Nx][(j-1)%Ny], vX[(i+1)%Nx][j], qubit_id=id, weight=wX, error_probability=pX)
            id=id+1
            Graph_Zerr.add_edge(vZ[i][j], vZ[i][(j + 1)%Ny], qubit_id=id, weight=wZ, error_probability=pZ)
            Graph_Xerr.add_edge(vX[i][j], vX[(i + 1)% Nx][j], qubit_id=id, weight=wX, error_probability=pX)
            id=id+1